Here we will collect all the data from the youtube API and save it to a csv file in the project root directory.

In [ ]:
# Import necessary libraries

import pandas as pd
from googleapiclient.discovery import build
from IPython.display import JSON
from dotenv import load_dotenv
import os
import isodate

In [ ]:
# Load API key from .env file

load_dotenv()
apiKey = os.getenv('API_KEY')


Here we will manually list all the youtube channels that we want to collect the data for. To do so we will need both the channel name and the channel ID

To find a youtube channel ID, you should:
1. go to the channel's youtube page, 
2. right click anywhere
3. select the option view page source
4. Press ctrl+f
5. search for ?channel_id

In [ ]:
# Dictionary of the youtube channels 
# from which we will retrieve the data.
# The key is the channel name and the 
# value is the ID

channelIDDict = {
    "gorgonoid": "UCLfCo17TCjx7qf-JMhQioLQ",
    "noticiasMaromba": "UCmK5h2-a4CquS4nIxDN6j7g",
    'albert': "UCij0YPRA_vGwKwCyn_o9MLw"
}

Now we will create the necessary functions to retrieve the data from the api.

In [ ]:
# Transforms the channel id dictionary to
# a string of the channel ID's separated
# by commas. Example: "id1, id2, id3"
def channelIDDictToCommaSeparatedIDString(channelIDDict):

    channelIDString = ""
    firstIDAdded = False
    for channel in channelIDDict:
        
        if firstIDAdded is False:
            channelIDString += channelIDDict[channel]
        else:
            channelIDString +=  ", " + channelIDDict[channel]

        firstIDAdded = True
    
    return channelIDString

In [ ]:
# Builds youtube api
def buildYoutubeAPI():

    api_service_name = "youtube"
    api_version = "v3"

    # Get credentials and create an API client
    youtube = build(
        api_service_name, api_version, developerKey=apiKey)

    return youtube

In [ ]:
# Gets the statistics of all channels
# included in the channel dictionary
def getAllChannelStatistics(channelIDDict): 

    allChannelStatistics = []
    
    request = buildYoutubeAPI().channels().list(
        part="snippet,contentDetails,statistics",
        id=channelIDDictToCommaSeparatedIDString(channelIDDict)
    )

    response = request.execute()

    # Loops through all channels and
    # adds data to dataframe
    for item in response['items']:
        channelStatistics = {
            "channelName": item['snippet']['title'],
            'subscriberCount': item['statistics']['subscriberCount'],
            'viewCount': item['statistics']['viewCount'],
            'videoCount': item['statistics']['videoCount'],
            'uploadedVideosPlaylistID': item['contentDetails']['relatedPlaylists']['uploads']
        }

        allChannelStatistics.append(channelStatistics)
        df = pd.DataFrame(allChannelStatistics)
    
    return df

Now that we have created getAllChannelStatistics() we can use it to retrieve information from the youtube channels we entered in the dictionary. Below, we will run a test to see if the channel information is being retrieved correctly.

In [ ]:
stats = getAllChannelStatistics(channelIDDict)

stats

In [ ]:
playlistID = 'UULfCo17TCjx7qf-JMhQioLQ'

def getUploadedVideoIDs(playlistID):

    videoIDs = []

    youtube = buildYoutubeAPI()

    next_page_token = None
    while True:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlistID,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response['items']:
            videoIDs.append(item['contentDetails']['videoId'])
    
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return videoIDs

videoIDs = getUploadedVideoIDs(playlistID)

len(videoIDs)

In [ ]:
def getVideoCategoriesBrazil():
    request = buildYoutubeAPI().videoCategories().list(
        part="snippet",
        regionCode='BR'
    )
    response = request.execute()

    categories = {}
    for item in response.get('items', []):
        id = item['id']
        title = item['snippet']['title']
        categories[id] = title
    
    return categories

categories = getVideoCategoriesBrazil()
print(categories)

In [ ]:
def getAllVideoStatistics(videoIDs):

    allVideoStatistics = []

    request = buildYoutubeAPI().videos().list(
        part="snippet,contentDetails,statistics",
        id=videoIDs[0:20]
    )
    response = request.execute()

    videoCategories = getVideoCategoriesBrazil()

    for item in response['items']:
        durationISO = item['contentDetails'].get('duration', None)
        if durationISO:
            durationTimeDelta = isodate.parse_duration(durationISO)
            durationSeconds = durationTimeDelta.total_seconds()
            hours = int(durationSeconds // 3600)
            minutes = int((durationSeconds % 3600) // 60)
            seconds = int(durationSeconds % 60)
            durationFormatted = f"{hours}h {minutes}m {seconds}s"
        else:
            durationFormatted = 'videoDuration unknown'

        videoCategoryID = item['snippet'].get('categoryId', None)

        videoCategory = ''
        if videoCategoryID:
            videoCategory = videoCategories[item['snippet'].get('categoryId', None)]
        else:
            videoCategory = 'videoCategoryID unknown'

        videoStatistics = {
            "videoPublishDatetime": item['snippet'].get('publishedAt', 'videoPublishDatetime unknown'),
            "channelID": item['snippet'].get('channelId', 'channelID unknown'),
            "videoTitle": item['snippet'].get('title', 'videoTitle unknown'),
            "videoDescription": item['snippet'].get('description', 'videoDescription unknown'),
            "channelTitle": item['snippet'].get('channelTitle', 'channelTitle unknown'),
            "videoTags": item['snippet'].get('tags', []),
            "videoCategory": videoCategory,
            "isLiveBroadcastContent": item['snippet'].get('liveBroadcastContent', 'isLiveBroadcastContent unknown'),
            "videoDuration": durationFormatted,
            "videoUploadStatus": item.get('status', {}).get('uploadStatus', 'videoUploadStatus unknown'),
            "isPublicStatsViewable": item.get('status', {}).get('publicStatsViewable', 'isPublicStatsViewable unknown'),
            "videoViewCount": item.get('statistics', {}).get('viewCount', 'videoViewCount unknown'),
            "videoLikeCount": item.get('statistics', {}).get('likeCount', 'videoLikeCount unknown'),
            "videoCommentCount": item.get('statistics', {}).get('commentCount', 'videoCommentCount unknown'),
            "videoRecordingLocation": item.get('recordingDetails', {}).get('location', 'videoRecordingLocation unknown'),
            "videoRecordingDatetime": item.get('recordingDetails', {}).get('recordingDate', 'videoRecordingDatetime unknown')
            
        }
        
        allVideoStatistics.append(videoStatistics)
        df = pd.DataFrame(allVideoStatistics)

    return df


videoStats = getAllVideoStatistics(videoIDs)

videoStats
